In [7]:
import cv2
import numpy as np
import cv2
import time
from IPython.display import clear_output
import math
from IPython.display import clear_output
    
#WIND_X = 480 
#WIND_Y = 360

WIND_X = 640
WIND_Y = 480

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def turn_to_RG (img):
    (h, w, d) = img.shape
    
    norm = np.zeros ((h, w), np.float)
    norm = img [:, :, 0].astype ('float') +\
           img [:, :, 1].astype ('float') +\
           img [:, :, 2].astype ('float')
    
    norm [norm == 0] = 5
    
    turned = np.zeros (img.shape, np.uint8)
    turned [:, :, 0] = ((img [:, :, 0].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 1] = ((img [:, :, 1].astype ('float')) / norm * 255).astype ('uint8')
    turned [:, :, 2] = ((img [:, :, 2].astype ('float')) / norm * 255).astype ('uint8')
    
    return turned

def obtain_color_ratio_mask (img, components, th, bl, use_rg = False, use_morph = False):
    sh = img [:, :, 0].shape
    
    rg = img
    
    if (use_rg == True):
        rg = turn_to_RG (img)
    
    smoothed = cv2.blur (rg, (bl, bl))
    
    needed = img.copy ()
    needed [:, :, 0] = np.full (sh, components [0])
    needed [:, :, 1] = np.full (sh, components [1])
    needed [:, :, 2] = np.full (sh, components [2])
    
    diff = cv2.absdiff (smoothed, needed)
    
    dif = diff [:, :, 1]# + diff [:, :, 1] + diff [:, :, 2]
    
    #cv2.imshow ("fok", dif)
    
    ret, res_mask = cv2.threshold (dif, th, 255, cv2.THRESH_BINARY_INV)
    
    #res_mask = cv2.adaptiveThreshold (dif, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 10)
    
    if (use_morph == True):
        res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_ERODE, np.ones ((int (bl), int (bl)), np.uint8))
        #res_mask = cv2.morphologyEx (res_mask, cv2.MORPH_OPEN, np.ones ((int (bl), int (bl)), np.uint8))
    
    #cv2.imshow ("kek", res_mask)
    
    res = cv2.bitwise_and (img, img, mask = res_mask)
    
    return res, res_mask

def find_max_bounding_box (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    return (left, top), (left + width, top + height)

def find_low_middle_point (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    print ("wid", width)
    
    return (left + int (width / 2), top + height)

def find_goal (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 255, 220), 30, 3, use_morph = True)    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (180, 220, 180), 30, 3, use_morph = True)    

    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    tl, br = find_max_bounding_box (goal_mask_res)
    return tl, br

def find_goal_ (img):
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    goal2, g_mask2 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = False)    
    
    cv2.imshow ("kek1", goal1)
    cv2.imshow ("kek2", goal2)

    #cv2.imshow ("mkek1", g_mask1)
    #cv2.imshow ("mkek2", g_mask2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    tl, br = find_max_bounding_box (goal_mask_res)
    return tl, br

def find_goal_middle (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 250, 220), 30, 3, use_morph = True)    
    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    mid = find_low_middle_point (goal_mask_res)
    return mid

#адаптивный трешхолд
#функция, на вход картинка на выход координаты середины
#написать, что надо застелить окрестность ворот
#



In [9]:
#"""
#cam = cv2.VideoCapture (0)
cam = cv2.VideoCapture ("inp.webm")

#cv2.namedWindow  ("field", cv2.WINDOW_NORMAL)
#cv2.resizeWindow ("field", (WIND_X, WIND_Y))

output_shape = (WIND_X * 3, WIND_Y * 2)

fourcc = cv2.VideoWriter_fourcc (*'XVID')
#out    = cv2.VideoWriter ('output.avi', fourcc, 20.0, output_shape)
#out    = cv2.VideoWriter ('output.mp4', fourcc, 20.0, output_shape)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

while (cam.isOpened ()):
    ret1, frame = cam.read ()
    
    if (ret1 != True):
        break
    
    frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))
    
    cv2.waitKey (1)
    
    tl, br = find_goal_ (frame_resized)
    goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
    
    ###mid = find_goal_middle (frame_resized)
    ###goal_on_inp_ = cv2.circle (frame_resized.copy (), mid, 5, (250, 10, 253), -1)
    
    goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.4, 0)
    ###goal_marked = cv2.addWeighted (goal_marked, 0.6, goal_on_inp_, 0.4, 0)
    
    #output_image = np.concatenate ((frame_resized, goal_res), axis = 1)
    output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)
    
    cv2.imshow ("goal detection", output_image)
    #out.write (output_image)
    
    #x0 =  1        #x coordinate of the point on the screen, pixels
    #y0 = -1.3      #y coordinate of the point on the screen, pixels
    
    #x0 = (tl [0] + br [0]) / 2 - WIND_X / 2
    #y0 = - tl [1] + WIND_Y / 2
    #k1 =  0      #incline  angle.               from  0 t0 1
    #k2 =  0      #rotation around Z axis angle. from -1 to 1
    
    #A = point_position(k1, k2, x0, y0)
    #B = ball_position(k1, k2, x0, y0)
    #clear_output (wait = True)
    #print (A)
    #print (x0, y0)
    
    time.sleep  (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord ('q')):
        break

cam.release ()
#out.release ()

cv2.waitKey           (1)
cv2.destroyAllWindows ()
#"""

In [18]:
def incline_dist (L, p, h, s, f = 600):
    #L - lense position
    #p - incline angle
    #h - object height
    #s - object pixel size
    #f - distance between the lense and the photomatrix
    
    sin = math.sin
    cos = math.cos
    
    b = s * cos (p) + f / sin (p)
    c = f * s * L [1] / (h * math.tan (p))
    
    print (b, "\n", c)
    
    d = b**2 - 4 * c
    w2 = (-b + math.sqrt (d)) / 2
    w1 = f / cos (p) + w2 * math.tan (p)
    w = np.array ([w1, w2], np.float)
    D = L - w * L [1] / w [1]
    
    return D

#3.179297910001

incline_dist (np.array ([7.1, 4.6], np.float), math.pi / 20 * 1, 5, 2.9778035, 2.7)

20.20076549558248 
 46.70196061458765


array([11.09361291,  0.        ])

In [28]:
output_shape = (WIND_X * 3, WIND_Y * 2)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

frame = cv2.imread ("5.jpg")

frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))

tl, br = find_goal (frame_resized)
#mid = find_goal_middle (frame_resized)

#print ("mid", mid [0] - WIND_X / 2, - mid [1] + WIND_Y / 2)

goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
#goal_on_inp = cv2.circle (frame_resized.copy (), mid, 8, (250, 10, 253), -1)

goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.7, 0)

output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)

cv2.imshow ("goal detection", output_image)

#L - lense position
#p - incline angle
#h - object height
#s - object pixel size
#f - distance between the lense and the photomatrix

#incline_dist (np.array ([7.1, 4.6], np.float), math.pi / 20 * 1, 5, 2.9778035, 2.7)

pix_sz = br [1] - tl [1]

print ("pixel size: ", pix_sz)

print (incline_dist (np.array ([7.1, 4.6], np.float), math.pi / 200, 90, pix_sz, 2.7))

time.sleep (0.01)

cv2.waitKey (0)

cv2.destroyAllWindows ()

pixel size:  113
284.8804667966292 
 992.6632216938194
[10.54839418  0.        ]


In [22]:
#x0 = (tl [0] + br [0]) / 2 - WIND_X / 2
#y0 = - tl [1] + WIND_Y / 2
#k1 =  0      #incline  angle.               from  0 t0 1
#k2 =  0      #rotation around Z axis angle. from -1 to 1
#A = point_position (k1, k2, x0, y0)
#print (A)
#print (x0, y0)